## Sales Prediction using a Deep Learning Algorithm(s)
### LSTM

In [16]:
import pickle
import mlflow
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn import preprocessing
from sklearn.model_selection import train_test_split 
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import MinMaxScaler 
from sklearn.metrics import mean_absolute_error
from time import gmtime, strftime
import matplotlib.pyplot as plt

import sys,os
sys.path.append(os.path.abspath(os.path.join('../scripts')))
import warnings
warnings.filterwarnings('ignore')

In [17]:
import dvc.api
import io
from ensurepip import version
from logger import Logger

logger = Logger("../logs/ml-prediction.log").get_app_logger()

In [18]:
def dvc_get_data(path: str, version: str, repo: str = '../'):
        try:
            content = dvc.api.read(path=path,
                                   repo=repo,
                                   rev=version)
            df = pd.read_csv(io.StringIO(content), sep=",")
            logger.info(
                f"DVC: CSV file read with path: {path} | version: {version} | from: {repo}")
            return df
        except:
            logger.exception("DVC data getter raised an exception")

In [19]:
data_path = '../data/cleaned_train.csv'
versions = 'cleaned-v'
repo = '../'
cleaned_df = dvc_get_data(path=data_path, version=versions, repo=repo)

cleaned_df.head()

,Store,DayOfWeek,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,CompetitionDistance,CompetitionOpenSinceMonth,...,Year,Month,Day,weekday,weekofyear,weekend,PromoInterval,Assortment,StoreType,Decile_rank
0,1,1,0.0,0.0,0,0,a,1,1270.0,9.0,...,2013,1,1,1,1,0,1,0,2,1
1,1,2,5530.0,668.0,1,0,0,1,1270.0,9.0,...,2013,1,2,2,1,0,1,0,2,1
2,1,3,4327.0,578.0,1,0,0,1,1270.0,9.0,...,2013,1,3,3,1,0,1,0,2,1
3,1,4,4486.0,619.0,1,0,0,1,1270.0,9.0,...,2013,1,4,4,1,0,1,0,2,1
4,1,5,4997.0,635.0,1,0,0,1,1270.0,9.0,...,2013,1,5,5,1,1,1,0,2,1


In [20]:
cleaned_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1017209 entries, 0 to 1017208
Data columns (total 24 columns):
 #   Column                     Non-Null Count    Dtype  
---  ------                     --------------    -----  
 0   Store                      1017209 non-null  int64  
 1   DayOfWeek                  1017209 non-null  int64  
 2   Sales                      1017209 non-null  float64
 3   Customers                  1017209 non-null  float64
 4   Open                       1017209 non-null  int64  
 5   Promo                      1017209 non-null  int64  
 6   StateHoliday               1017209 non-null  object 
 7   SchoolHoliday              1017209 non-null  int64  
 8   CompetitionDistance        1017209 non-null  float64
 9   CompetitionOpenSinceMonth  1017209 non-null  float64
 10  CompetitionOpenSinceYear   1017209 non-null  float64
 11  Promo2                     1017209 non-null  int64  
 12  Promo2SinceWeek            1017209 non-null  float64
 13  Promo2SinceY

In [21]:
cleaned_df.shape

(1017209, 24)